In [ ]:
import numpy as np
import adsorbate_thermo
import matplotlib.pyplot as plt
import rmgpy.species
import copy

In [ ]:
molecular_weight = 18.02
frequencies = [49.5, 68.6, 73.6, 102.0, 437.6, 452.9, 1596.3, 3675.6, 3787.0]
composition = {'H': 2, 'O': 1, 'C': 0, 'N': 0}
heat_of_formation_0K = -259.05  # kJ/mol

my_calc = adsorbate_thermo.AdsorbateThermoCalc(molecular_weight, frequencies, composition, heat_of_formation_0K, twoD_gas=True)


In [ ]:
nasa = my_calc.get_thermo()
a_low, a_high = my_calc.get_thermo2()

In [ ]:
def get_thermo_from_NASA2(temperatures, a_low, a_high):
    # compute thermo properties from nasa polynomials
    T_switch = 1000.0
    R = rmgpy.constants.R
    i_switch = -1
    for i in range(len(temperatures)):
        if temperatures[i] == T_switch:
            i_switch = i

    cp_fit = np.zeros(len(temperatures))
    h_fit = np.zeros(len(temperatures))
    s_fit = np.zeros(len(temperatures))
    for (i, temp) in enumerate(temperatures):
        if temp <= T_switch:
            cp_fit[i] = a_low[0] + a_low[1] * temp + a_low[2] * temp**2.0 + a_low[3] * temp**3.0 + a_low[4] * temp**4.0
            h_fit[i] = a_low[0] * temp + a_low[1] / 2.0 * temp**2.0 + a_low[2] / 3.0 * temp**3.0 + a_low[3] / 4.0 * temp**4.0 + a_low[4] / 5.0 * temp**5.0 + a_low[5]
            s_fit[i] = a_low[0] * np.log(temp) + a_low[1] * temp + a_low[2] / 2.0 * temp**2.0 + a_low[3] / 3.0 * temp**3.0 + a_low[4] / 4.0 * temp**4.0 + a_low[6]
        else:
            cp_fit[i] = a_high[0] + a_high[1] * temp + a_high[2] * temp**2.0 + a_high[3] * temp**3.0 + a_high[4] * temp**4.0
            h_fit[i] = a_high[0] * temp + a_high[1] / 2.0 * temp**2.0 + a_high[2] / 3.0 * temp**3.0 + a_high[3] / 4.0 * temp**4.0 + a_high[4] / 5.0 * temp**5.0 + a_high[5]
            s_fit[i] = a_high[0] * np.log(temp) + a_high[1] * temp + a_high[2] / 2.0 * temp**2.0 + a_high[3] / 3.0 * temp**3.0 + a_high[4] / 4.0 * temp**4.0 + a_high[6]

    cp_fit *= R
    h_fit *= R
    s_fit *= R

    return cp_fit, h_fit, s_fit


In [ ]:
a_low0 = nasa.polynomials[0].coeffs
a_high0 = nasa.polynomials[1].coeffs
cp_fit, h_fit, s_fit = my_calc.get_thermo_from_NASA2(a_low, a_high)
cp_fit0, h_fit0, s_fit0 = my_calc.get_thermo_from_NASA2(a_low0, a_high0)

In [ ]:
# check rmg reading of Katrin's NASA polynomials
nasa2 = copy.deepcopy(nasa)
nasa2.polynomials[0] = a_low
nasa2.polynomials[1] = a_high

H1s = np.zeros(len(my_calc.temperatures))
Cp1s = np.zeros(len(my_calc.temperatures))
S1s = np.zeros(len(my_calc.temperatures))
H2s = np.zeros(len(my_calc.temperatures))
Cp2s = np.zeros(len(my_calc.temperatures))
S2s = np.zeros(len(my_calc.temperatures))

for i, T in enumerate(my_calc.temperatures):
    H1s[i] = nasa.get_enthalpy(T) / 1000.0  # units are in J/mol and we want kJ/mol
    Cp1s[i] = nasa.get_heat_capacity(T) / 1000.0
    S1s[i] = nasa.get_heat_capacity(T) / 1000.0
    H2s[i] = nasa2.get_enthalpy(T) / 1000.0  # units are in J/mol and we want kJ/mol
    Cp2s[i] = nasa2.get_heat_capacity(T) / 1000.0
    S2s[i] = nasa2.get_heat_capacity(T) / 1000.0

In [ ]:
plt.plot(my_calc.temperatures, cp_fit, label='Katrin NASA and get_thermo')
plt.plot(my_calc.temperatures, Cp1s, label='RMG NASA and get_thermo')
plt.plot(my_calc.temperatures, cp_fit0/1000, label='RMG NASA and Katrin get_thermo')
plt.plot(my_calc.temperatures, Cp2s, label='Katrin NASA and RMG get_thermo')
plt.title('Heat Capacity')
plt.legend()

In [ ]:
plt.plot(my_calc.temperatures, h_fit, label='Katrin NASA and get_thermo')
plt.plot(my_calc.temperatures, H1s, label='RMG NASA and get_thermo')
plt.plot(my_calc.temperatures, h_fit0/1000, label='RMG NASA and Katrin get_thermo')
plt.plot(my_calc.temperatures, H2s, label='Katrin NASA and RMG get_thermo')
plt.title('Enthalpy Capacity')
plt.legend()

In [ ]:
plt.plot(my_calc.temperatures, S1s, label='RMG NASA and get_thermo')
plt.plot(my_calc.temperatures, S2s, label='Katrin NASA and RMG get_thermo')
plt.plot(my_calc.temperatures, s_fit, label='Katrin NASA and get_thermo')
plt.plot(my_calc.temperatures, s_fit0/1000, label='RMG NASA and Katrin get_thermo')
plt.title('Entropy')
plt.legend()

In [ ]:
plt.plot(my_calc.temperatures, s_fit, label='Katrin NASA and get_thermo')
plt.plot(my_calc.temperatures, s_fit0/1000, label='RMG NASA and Katrin get_thermo')
plt.title('Entropy')
plt.legend()

In [ ]:
plt.plot(my_calc.temperatures, S1s, label='RMG NASA and get_thermo')
plt.plot(my_calc.temperatures, S2s, label='Katrin NASA and RMG get_thermo')
plt.title('Entropy')
plt.legend()